# Concat Operator

* https://xarray.pydata.org/en/stable/user-guide/combining.html

In [ ]:
import xarray as xr
import pandas as pd

In [ ]:
from pathlib import Path
data_dir = Path("/Users/pingu/data/cmip6-decadal")

In [ ]:
cmip6_files = list(data_dir.glob('**/*.nc'))
cmip6_files.sort()
cmip6_files

In [ ]:
ds0 = xr.open_dataset(cmip6_files[0])
ds0

In [ ]:
ds1 = xr.open_dataset(cmip6_files[0])
ds1

In [ ]:
ds_concat = xr.concat([ds0, ds1], dim=pd.Index([0, 1], name="realization"))
ds_concat

In [ ]:
ds_concat.realization

## use open_mfdataset

In [ ]:
ds_agg = xr.open_mfdataset(cmip6_files, concat_dim="realization", combine="nested")
ds_agg

In [ ]:
ds_agg.isel(realization=0)